In [1]:
#@title Mount your Google Drive
%matplotlib inline
%load_ext autoreload
%autoreload 2

from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [2]:
#@markdown Enter the path to the assignment folder in your Google Drive
import sys
import os
import shutil
import warnings

folder = "/content/gdrive/MyDrive/ELE6310_project/" #@param {type:"string"}

!ln -Ts $folder /content/A1 2> /dev/null

# Add the assignment folder to Python path
if '/content/gdrive/MyDrive/ELE6310_project/' not in sys.path:
    sys.path.insert(0, '/content/gdrive/MyDrive/ELE6310_project/')

# Install requirements
#!pip install -qr /content/A1/requirements.txt
!pip install -qr   /content/gdrive/MyDrive/ELE6310_project/requirements.txt


# Check if CUDA is available
import torch
if not torch.cuda.is_available():
    warnings.warn('CUDA is not available.')
else:
  print("cuda is available")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 357.3/357.3 KB 7.7 MB/s eta 0:00:00
cuda is available


In [3]:
import torch
import numpy as np
import random
from matplotlib import pyplot as plt
import os

In [4]:
from common.utils import load_CIFAR10_dataset, evaluate, fit, model_size
from common import quant_resnet
from common import train_eval

In [5]:
Seed = 6310
torch.manual_seed(Seed)
np.random.seed(Seed)
random.seed(Seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(Seed)
    torch.cuda.manual_seed_all(Seed)

In [6]:
model = quant_resnet.CifarResNet(block = quant_resnet.BasicBlock, layers= [3,3,3])
device = torch.device('cuda:0')
model.to(device)
train_loader, test_loader, calibration_loader = load_CIFAR10_dataset(batch_size=256, calibration_batch_size=1024)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [7]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), 1e-3, momentum=0.9, weight_decay=0.0005, nesterov=True)

In [8]:
for epoch in range(100):
    train_acc, train_loss = train_eval.train(model, train_loader, optimizer, criterion, device)
    test_acc, test_loss = train_eval.evaluate(model, test_loader, criterion, device)
    print(f"Epoch {epoch+1}: train_loss={train_loss:.4f}, train_acc={train_acc:.2f}%, test_loss={test_loss:.4f}, test_acc={test_acc:.2f}%")

Epoch 1: train_loss=2.2370, train_acc=16.61%, test_loss=2.1130, test_acc=23.40%
Epoch 2: train_loss=2.0179, train_acc=24.91%, test_loss=1.8937, test_acc=27.84%
Epoch 3: train_loss=1.8585, train_acc=30.19%, test_loss=1.8029, test_acc=32.26%
Epoch 4: train_loss=1.7795, train_acc=32.73%, test_loss=1.7379, test_acc=33.41%
Epoch 5: train_loss=1.7256, train_acc=34.23%, test_loss=1.7353, test_acc=34.20%
Epoch 6: train_loss=1.6747, train_acc=36.26%, test_loss=1.7239, test_acc=35.34%
Epoch 7: train_loss=1.6387, train_acc=37.43%, test_loss=1.6195, test_acc=38.36%
Epoch 8: train_loss=1.6072, train_acc=38.85%, test_loss=1.5561, test_acc=41.04%
Epoch 9: train_loss=1.5785, train_acc=40.06%, test_loss=1.6335, test_acc=38.32%
Epoch 10: train_loss=1.5511, train_acc=41.50%, test_loss=1.5647, test_acc=41.70%
Epoch 11: train_loss=1.5293, train_acc=42.59%, test_loss=1.5053, test_acc=43.20%
Epoch 12: train_loss=1.5031, train_acc=43.86%, test_loss=1.4634, test_acc=45.35%
Epoch 13: train_loss=1.4850, train_ac

In [9]:
torch.save(model.state_dict(), 'Quant_model_W3A8_3.pt')